In [18]:
import google.generativeai as genai
import os
from datetime import datetime
import re


In [19]:
# Load the API key from environment variables and configure the model properly
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')  # Make sure you have set this environment variable
if not GOOGLE_API_KEY:
    raise ValueError("Please set the GOOGLE_API_KEY environment variable")

genai.configure(api_key=GOOGLE_API_KEY)

# Initialize the model with safety settings
generation_config = {
    "temperature": 0.7,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 2048,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",  # or "gemini-1.5-pro-latest"
    generation_config=generation_config,
    safety_settings=safety_settings
)

In [ ]:
client_sim_prompt = """
    You are a client looking for a data analyst to solve a business problem.
    Generate a client profile and the details of a realistic business problem that I need solved.
    As the client, describe your personality, background, your understanding of data analysis, and any specific quirks or communication styles you might have, make it somewhat unpredictable.
    Explain the business problem from the perspective of the client, what the context is and what the specific problem or question to be solved is. Include fictional demands, caveats, and surprises that are relevant to me as the client.
    Speak in first person.
    """

In [20]:
# Global variable to store the project documentation.
documentation_log = ""

def log_documentation(text):
    global documentation_log
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}]: {text}\n"
    documentation_log += log_entry

def generate_client_and_problem(model):
    prompt = client_sim_prompt
    response = model.generate_content(prompt)
    log_documentation(f"Generated client and problem with prompt: {prompt}")
    return response.text

def extract_client_name(model, text):
  prompt = f"""
  From the following text, extract the client's name, return only the name:
  {text}
  """
  response = model.generate_content(prompt)
  log_documentation(f"Extracting client name with prompt: {prompt}")
  return response.text.strip()

def main():
    log_documentation("Starting the data analysis project simulation.")
    client_and_problem = generate_client_and_problem(model)
    
    # Extract the client name
    client_name = extract_client_name(model,client_and_problem)
    
    # Sanitize the client name for use in a file name
    sanitized_client_name = re.sub(r'[^\w\s-]', '', client_name)
    sanitized_client_name = re.sub(r'\s+', '_', sanitized_client_name).strip()


    # Create the 'simulated_clients' directory if it doesn't exist
    clients_dir = "simulated_clients"
    if not os.path.exists(clients_dir):
        os.makedirs(clients_dir)
    
    # Create a unique filename
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join(clients_dir, f"{sanitized_client_name}_{timestamp}.txt")

    # Save the client and problem
    with open(filename, "w") as f:
         f.write(client_and_problem)
         
    print(f"Client Profile and Business Problem saved to:\n{filename}")

    # Save documentation
    with open("project_documentation.txt", "w") as f:
        f.write(documentation_log)
    print("\nProject documentation saved to project_documentation.txt")
    log_documentation("Project simulation completed.")

In [21]:

if __name__ == "__main__":
    main()

Client Profile and Business Problem saved to:
simulated_clients\Amelia_Dubois_20250110_212926.txt

Project documentation saved to project_documentation.txt
